### =======================================================================
### IMPORTACIÓN DE DATOS.
### =======================================================================

In [1]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.cross_validation import train_test_split
import xgboost as xgb

%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)
pd.set_option('display.max_columns', 1000)
np.set_printoptions(threshold=np.nan)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES EVENTOS.
eventos = pd.read_csv('events_up_to_01062018.csv', encoding = 'utf-8')

### =======================================================================
### ARMADO DE FEATURES.
### =======================================================================\

In [3]:
## PASAMOS LAS COLUMNAS QUE TIENEN UNA CANTIDAD DE VALORES LIMITADA A UN TIPO CATEGORY
eventos['event'] = eventos['event'].astype('category')
eventos['search_engine'] = eventos['search_engine'].astype('category')
eventos['channel'] = eventos['channel'].astype('category')
eventos['new_vs_returning'] = eventos['new_vs_returning'].astype('category')
eventos['device_type'] = eventos['device_type'].astype('category')
eventos['color'] = eventos['color'].astype('category')
eventos['region'] = eventos['region'].astype('category')
eventos['city'] = eventos['city'].astype('category')
eventos['browser_version'] = eventos['browser_version'].astype('category')
eventos['screen_resolution'] = eventos['screen_resolution'].astype('category')
eventos['timestamp'] = pd.to_datetime(eventos['timestamp'], infer_datetime_format=True)

In [4]:
## ORDENAMOS LOS DATOS ṔOR PERSONAS EN PRIMER LUGAR Y TIEMPO EN SEGUNDO.
eventos.sort_values(['person', 'timestamp'], ascending=[True, True], inplace=True)

In [5]:
## DIFERENCIAMOS EN TRES COLUMNAS DIFERENTES EL DIA, MES Y AÑO.
eventos['mes'] = eventos['timestamp'].dt.month
eventos['dia'] = eventos['timestamp'].dt.day
eventos['hora'] = eventos['timestamp'].dt.hour

In [6]:
## ARMAMOS UNA COLUMNA PARA EL DÍA DE LA SEMANA COMO NOMBRE.
eventos['diasemana'] = eventos['timestamp'].dt.weekday_name
## PONEMOS LOS NOMBRES DE MANERA MÁS PROLIJA PARA LOS GRÁFICOS.
eventos.loc[eventos.diasemana.str.contains('Monday', na=False), 'diasemana'] = 'lunes'
eventos.loc[eventos.diasemana.str.contains('Tuesday', na=False), 'diasemana'] = 'martes'
eventos.loc[eventos.diasemana.str.contains('Wednesday', na=False), 'diasemana'] = 'miercoles'
eventos.loc[eventos.diasemana.str.contains('Thursday', na=False), 'diasemana'] = 'jueves'
eventos.loc[eventos.diasemana.str.contains('Friday', na=False), 'diasemana'] = 'viernes'
eventos.loc[eventos.diasemana.str.contains('Saturday', na=False), 'diasemana'] = 'sabado'
eventos.loc[eventos.diasemana.str.contains('Sunday', na=False), 'diasemana'] = 'domingo'

In [7]:
# AGREGAMOS UNA COLUMNA PARA INDICAR SI EL EVENTO OCURRIO UN FIN DE SEMANA
eventos['evento_en_finde'] = 0
eventos.loc[(eventos.diasemana.str.contains('DOM', na=False) | eventos.diasemana.str.contains('SAB', na=False)), 'evento_en_finde'] = 1

In [8]:
# DEFINIMOS EL MES COMO NOMBRE PARA FACILITAR LAS COLUMNAS
eventos['mesMayus'] = ''
eventos.loc[eventos.mes == 1, 'mesMayus'] = 'enero'
eventos.loc[eventos.mes == 2, 'mesMayus'] = 'febrero'
eventos.loc[eventos.mes == 3, 'mesMayus'] = 'marzo'
eventos.loc[eventos.mes == 4, 'mesMayus'] = 'abril'
eventos.loc[eventos.mes == 5, 'mesMayus'] = 'mayo'
eventos.loc[eventos.mes == 6, 'mesMayus'] = 'junio'
eventos.loc[eventos.mes == 7, 'mesMayus'] = 'julio'
eventos.loc[eventos.mes == 8, 'mesMayus'] = 'agosto'
eventos.loc[eventos.mes == 9, 'mesMayus'] = 'septiembre'
eventos.loc[eventos.mes == 10, 'mesMayus'] = 'octubre'
eventos.loc[eventos.mes == 11, 'mesMayus'] = 'noviembre'
eventos.loc[eventos.mes == 12, 'mesMayus'] = 'diciembre'

In [9]:
# ARMAMOS UNA LÓGICA PARA SEGMENTAR LAS FRANJAS HORARIAS.
# MADRUGADA de 00 a 06
eventos['hora_madrugada'] = 0
eventos.loc[((eventos.hora > -1) & (eventos.hora < 7)), 'hora_madrugada'] = 1
# MAÑANA de 07 a 11
eventos['hora_maniana'] = 0
eventos.loc[((eventos.hora > 6) & (eventos.hora < 12)), 'hora_maniana'] = 1
# ALMUERZO de 12 a 13
eventos['hora_almuerzo'] = 0
eventos.loc[((eventos.hora > 11) & (eventos.hora < 14)), 'hora_almuerzo'] = 1
# TARDE de 14 a 18
eventos['hora_tarde'] = 0
eventos.loc[((eventos.hora > 13) & (eventos.hora < 19)), 'hora_tarde'] = 1
# NOCHE de 19 a 23
eventos['hora_noche'] = 0
eventos.loc[((eventos.hora > 18) & (eventos.hora < 24)), 'hora_noche'] = 1
# TRANSFORMAMOS EN CATEGÓRICAS EL DÍA DE LA SEMANA Y EL MES.
eventos['diasemana'] = eventos['diasemana'].astype('category')
eventos['mesMayus'] = eventos['mesMayus'].astype('category')

In [10]:
## CREAMOS UNA COLUMNA CON CONTENIDO VACIO.
eventos['sistema'] = 'OtrosSistemas'
## SEGÚN QUE FAMILIA DE SO POSEA ACTUALIZAMOS NUESTRA NUEVA COLUMNA.
eventos.loc[eventos.operating_system_version.str.contains('Mac', na=False), 'sistema'] = 'MacOS'
eventos.loc[eventos.operating_system_version.str.contains('iOS', na=False), 'sistema'] = 'iOS'
eventos.loc[eventos.operating_system_version.str.contains('Chrome', na=False), 'sistema'] = 'Chrome'
eventos.loc[eventos.operating_system_version.str.contains('Tizen', na=False), 'sistema'] = 'Tizen'
eventos.loc[eventos.operating_system_version.str.contains('Android', na=False), 'sistema'] = 'Android'
eventos.loc[eventos.operating_system_version.str.contains('Windows Phone', na=False), 'sistema'] = 'Windows Phone'
eventos.loc[eventos.operating_system_version.str.contains('Ubuntu', na=False), 'sistema'] = 'Ubuntu'
eventos.loc[eventos.operating_system_version.str.contains('Linux', na=False), 'sistema'] = 'Linux'
## CASO PARTICULAR, WINDOWS Y WINDOWS PHONE COMPARTEN LA PALABRA.
eventos.loc[(eventos.operating_system_version.str.contains('Windows', na=False) & ~eventos.operating_system_version.str.contains('Phone', na=False)), 'sistema'] = 'Windows'
## AHORA SI PASAMOS LA COLUMNA DE SISTEMAS OPERATIVOS A UN ENUMERATIVO.
eventos['operating_system_version'] = eventos['operating_system_version'].astype('category')
eventos['sistema'] = eventos['sistema'].astype('category')

In [11]:
## AGREGAMOS UNA COLUMNA PARA DIFERENCIAR LAS MARCAS.
eventos['marca'] = 'OtrasMarcas'
## VAMOS OBTENIENDO LAS MARCAS SEGÚN LAS PALABRAS CLAVES QUE LAS CONFORMAN.
eventos.loc[eventos.model.str.contains('Samsung', na=False), 'marca'] = 'Samsung'
eventos.loc[eventos.model.str.contains('Motorola', na=False), 'marca'] = 'Motorola'
eventos.loc[eventos.model.str.contains('Sony', na=False), 'marca'] = 'Sony'
eventos.loc[eventos.model.str.contains('LG ', na=False), 'marca'] = 'LG'
eventos.loc[eventos.model.str.contains('iPad', na=False), 'marca'] = 'iPad'
eventos.loc[eventos.model.str.contains('Asus', na=False), 'marca'] = 'Asus'
eventos.loc[eventos.model.str.contains('iPhone', na=False), 'marca'] = 'iPhone'
eventos.loc[eventos.model.str.contains('Quantum', na=False), 'marca'] = 'Quantum'
eventos.loc[eventos.model.str.contains('Lenovo', na=False), 'marca'] = 'Lenovo'
## AHORA SI PASAMOS LA COLUMNA DE MODELOS A UN ENUMERATIVO.
eventos['model'] = eventos['model'].astype('category')
eventos['marca'] = eventos['marca'].astype('category')

In [12]:
## AGREGAMOS UNA COLUMNA PARA DIFERENCIAR BRASIL DEL RESTO.
eventos['pais'] = 'OtrosPaises'
eventos.loc[eventos.country.str.contains('Brazil', na=False), 'pais'] = 'Brasil'
## AHORA SI PASAMOS LA COLUMNA DE PAISES A UN ENUMERATIVO.
eventos['pais'] = eventos['pais'].astype('category')
eventos['country'] = eventos['country'].astype('category')

In [13]:
eventos['Capacidad_en_GB'] = 0
eventos.loc[eventos.storage.str.contains('16GB', na=False), 'Capacidad_en_GB'] = 16
eventos.loc[eventos.storage.str.contains('32GB', na=False), 'Capacidad_en_GB'] = 32
eventos.loc[eventos.storage.str.contains('64GB', na=False), 'Capacidad_en_GB'] = 64
eventos.loc[eventos.storage.str.contains('128GB', na=False), 'Capacidad_en_GB'] = 128
eventos.loc[eventos.storage.str.contains('8GB', na=False), 'Capacidad_en_GB'] = 8
eventos.loc[eventos.storage.str.contains('256GB', na=False), 'Capacidad_en_GB'] = 256
eventos.loc[eventos.storage.str.contains('4GB', na=False), 'Capacidad_en_GB'] = 4
eventos.loc[eventos.storage.str.contains('512MB', na=False), 'Capacidad_en_GB'] = 0.512
eventos['storage'] = eventos['storage'].astype('category')

In [14]:
## CREAMOS UNA COLUMNA PARA DIFERENCIAR LA CONDICION.
eventos['Puntaje'] = 0
## ABSTRAEMOS SOLO AQUELLOS QUE TIENEN UNA CONDICIÓN ASOCIADA Y LO PASAMOS A NUESTRO IDIOMA.
eventos.loc[eventos.condition.str.contains('Excelente', na=False), 'Puntaje'] = 4
eventos.loc[eventos.condition.str.contains('Muito Bom', na=False), 'Puntaje'] = 3
eventos.loc[eventos.condition.str.contains('Novo', na=False), 'Puntaje'] = 5
## SOLO NOS QUEDAN DOS TIPOS DE EVENTOS QUE SON REFERENCIA A LOS 'BUENOS'
eventos.loc[eventos.Puntaje == 0, 'Puntaje'] = 2
eventos['condition'] = eventos['condition'].astype('category')

In [15]:
# COMO TENEMOS UN EVENTO CON EL MISMO NOMBRE SE GENERA CONFLICTOS, 
# ERGO LE MODIFICAMOS EL NOMBRE PARA NO TENER DOS COLUMNAS CON = NOMBRE Y DISTINTO TIPO.
eventos.rename(columns={'staticpage': 'Genstatpage'}, inplace=True)

In [16]:
# COLUMNAS DONDE POR CADA REGISTRO SABEMOS QUE TENEMOS UN VALOR (SIEMPRE PRESENTES)
dummies = pd.get_dummies(eventos['diasemana'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
dummies = pd.get_dummies(eventos['mesMayus'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
dummies = pd.get_dummies(eventos['event'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
dummies = pd.get_dummies(eventos['storage'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
dummies = pd.get_dummies(eventos['condition'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
dummies = pd.get_dummies(eventos['new_vs_returning'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
dummies = pd.get_dummies(eventos['sistema'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
dummies = pd.get_dummies(eventos['marca'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
dummies = pd.get_dummies(eventos['pais'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)
dummies = pd.get_dummies(eventos['device_type'], drop_first=False)
eventos = pd.concat([eventos, dummies], axis=1)

In [17]:
eventos.rename(columns={'generic listing': 'geneList', 'staticpage': 'statpage', 'staticpage': 'SP'}, inplace=True)

In [18]:
eventos.columns

Index([  u'timestamp',       u'event',      u'person',         u'url',
               u'sku',       u'model',   u'condition',     u'storage',
             u'color',        u'skus',
       ...
           u'Samsung',        u'Sony',        u'iPad',      u'iPhone',
            u'Brasil', u'OtrosPaises',    u'Computer',  u'Smartphone',
            u'Tablet',     u'Unknown'],
      dtype='object', length=103)

In [19]:
columnas_relevantes = list(eventos.select_dtypes(include=['int','float64','uint8']).columns)

columnas_relevantes.remove('sku')
columnas_relevantes.append('person')
eventos_filtrados = eventos.loc[:, eventos.columns.isin(columnas_relevantes)]
columnas_relevantes.remove('person')

eventos_por_usuario = eventos_filtrados.groupby('person')[columnas_relevantes].sum().astype(np.float16).reset_index()

### =======================================================================
### ENTRENAMIENTO Y PREDICCIÓN.
### =======================================================================

In [20]:
## OBTENEMOS TODA LA INFORMACIÓN DEL SET DE ENTRENAMIENTO.
y_train = pd.read_csv('labels_training_set.csv', encoding = 'utf-8')
test_users = pd.read_csv('trocafone_kaggle_test.csv', encoding = 'utf-8')

In [21]:
## Filtramos los eventos para los usuarios que se encuentran en el set de entrenamiento
train = pd.merge(eventos_por_usuario, y_train, on='person', how='inner')
test = pd.merge(eventos_por_usuario, test_users)

In [22]:
features = list(columnas_relevantes)
x = train[features]
y = train['label']

In [23]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y)

In [24]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

xtest.head()

(14560, 75)
(14560,)
(4854, 75)
(4854,)


,mes,dia,hora,evento_en_finde,hora_madrugada,hora_maniana,hora_almuerzo,hora_tarde,hora_noche,Capacidad_en_GB,Puntaje,domingo,jueves,lunes,martes,miercoles,sabado,viernes,abril,enero,febrero,marzo,mayo,ad campaign hit,brand listing,checkout,conversion,geneList,lead,search engine hit,searched products,SP,viewed product,visited site,128GB,16GB,256GB,32GB,4GB,512MB,64GB,8GB,Bom,Bom - Sem Touch ID,Excelente,Muito Bom,Novo,New,Returning,Android,Chrome,Linux,MacOS,OtrosSistemas,Tizen,Ubuntu,Windows,Windows Phone,iOS,Asus,LG,Lenovo,Motorola,OtrasMarcas,Quantum,Samsung,Sony,iPad,iPhone,Brasil,OtrosPaises,Computer,Smartphone,Tablet,Unknown
14877,4.425781,18.875000,13.421875,0.0,0.2771,0.0,0.186768,0.096375,0.439697,8.00,2.427734,0.180664,0.47583,0.024094,0.0,0.0,0.246948,0.072266,0.331299,0.0,0.0,0.120483,0.54834,0.084351,0.012047,0.030121,0.0,0.066284,0.0,0.030121,0.210815,0.0,0.475830,0.090332,0.072266,0.066284,0.000000,0.174683,0.0,0.0,0.192749,0.0,0.198853,0.0,0.120483,0.186768,0.0,0.006023,0.084351,0.054230,0.0,0.0,0.0,0.909668,0.0,0.0,0.036133,0.0,0.000000,0.0,0.00,0.0,0.0,0.493896,0.0,0.048187,0.0,0.0,0.457764,0.090332,0.909668,0.036133,0.054230,0.000000,0.0
7594,5.000000,22.000000,20.000000,0.0,0.0000,0.0,0.000000,0.000000,1.000000,2.00,2.500000,0.000000,0.00000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.00000,0.250000,0.000000,0.250000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.500000,0.0,0.000000,0.0,0.000000,0.500000,0.0,0.250000,0.000000,0.250000,0.0,0.0,0.0,0.750000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.0,0.500000,0.0,0.000000,0.0,0.0,0.500000,0.250000,0.750000,0.000000,0.250000,0.000000,0.0
18637,5.000000,19.000000,18.000000,0.0,0.0000,0.0,0.000000,1.000000,0.000000,8.00,2.250000,0.000000,0.00000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.000000,1.00000,0.166626,0.250000,0.083313,0.0,0.083313,0.0,0.166626,0.000000,0.0,0.166626,0.083313,0.000000,0.000000,0.000000,0.250000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.250000,0.0,0.083313,0.000000,0.083313,0.0,0.0,0.0,0.916504,0.0,0.0,0.000000,0.0,0.000000,0.0,0.25,0.0,0.0,0.750000,0.0,0.000000,0.0,0.0,0.000000,0.083313,0.916504,0.000000,0.000000,0.083313,0.0
9927,5.000000,25.000000,18.000000,0.0,0.0000,0.0,0.000000,1.000000,0.000000,4.00,2.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.000000,1.00000,0.250000,0.000000,0.250000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.5,0.500000,0.0,0.000000,0.000000,0.0,0.250000,0.000000,0.250000,0.0,0.0,0.0,0.750000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.0,0.500000,0.0,0.500000,0.0,0.0,0.000000,0.250000,0.750000,0.000000,0.250000,0.000000,0.0
3067,5.000000,19.890625,18.062500,0.0,0.0000,0.0,0.000000,0.722168,0.277832,106.25,2.945312,0.000000,0.00000,0.444336,0.0,0.0,0.555664,0.000000,0.000000,0.0,0.0,0.000000,1.00000,0.000000,0.000000,0.222168,0.0,0.055542,0.0,0.000000,0.000000,0.0,0.500000,0.222168,0.166626,0.000000,0.388916,0.166626,0.0,0.0,0.000000,0.0,0.000000,0.0,0.222168,0.500000,0.0,0.055542,0.166626,0.000000,0.0,0.0,0.0,0.777832,0.0,0.0,0.000000,0.0,0.222168,0.0,0.00,0.0,0.0,0.277832,0.0,0.000000,0.0,0.0,0.722168,0.222168,0.777832,0.000000,0.222168,0.000000,0.0


In [26]:
for estimators in range(195,201):
    #for depth in range(1,5):
    xgboost = xgb.XGBClassifier(n_estimators=estimators, max_depth=5, learning_rate=0.05)
    xgboost.fit(xtrain, ytrain)
    y_pred_rf = xgboost.predict_proba(xtest)[:,1]
    print('estimators:' + str(estimators) + str(np.sqrt(metrics.mean_squared_error(y_pred_rf, ytest))))

estimators:1950.21448059079101625
estimators:1960.21449626842341835
estimators:1970.21448488099110144
estimators:1980.21446981649842808
estimators:1990.21447951223553108
estimators:2000.21448888172531547
estimators:2010.21449117918376787
estimators:2020.21447779353881577
estimators:2030.21446224460797664
estimators:2040.21446996873288554


In [27]:
xgboost = xgb.XGBClassifier(n_estimators=200, learning_rate=0.05)
xgboost.fit(x, y)
y_pred_rf = xgboost.predict_proba(test[features])[:,1]

In [28]:
## =================================================================================================
## ARMAMOS EN BASE A LA PREDICCIÓN QUE TENEMOS UN CSV PARA SUBIR A KAGGLE CON EL FORMATO INDICADO.
## =================================================================================================
submission = pd.DataFrame({ 'label': y_pred_rf, 'person': test['person'] })
submission.to_csv("submission_grupo17_XGB_06.csv", index=False)

In [ ]:
## SE AGREGÓ EL FEATURE QUE ENGLOBA LOS SISTEMAS OPERATIVOS MÁS COMUNES UTILIZADOS O BUSCADOS POR LOS USUARIOS:
## PREVIAMENTE TENÍAMOS: 0.84077
## RESULTADO DEL SUBMIT: 0.85216 >> VEMOS UNA MEJORA.

In [ ]:
## SE AGREGÓ EL FEATURE QUE ENGLOBA LOS MODELOS MÁS COMUNES UTILIZADOS O BUSCADOS POR LOS USUARIOS:
## PREVIAMENTE TENÍAMOS: 0.85216
## RESULTADO DEL SUBMIT:0.85294 >> VEMOS UNA MEJORA.

In [ ]:
## SE AGREGÓ UN PAR DE FEATURES (pais, device_type, Capacidad_en_GB, Puntaje):
## PREVIAMENTE TENÍAMOS: 0.85294
## RESULTADO DEL SUBMIT: 0.85524 >> VEMOS UNA MEJORA.

In [ ]:
## AHORA LO QUE SE BUSCA ES HACER EL PROMEDIO EN LUGAR DE LA SUMA:
## PREVIAMENTE TENÍAMOS: 0.85524
## RESULTADO DEL SUBMIT: 0.85519 >> CON LO CUAL EMPEORA.